<a href="https://colab.research.google.com/github/lisahqwang/Reinforcement-Learning-with-Crypto-Data/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%matplotlib inline

import os
import random
import json
import glob
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt

from collections import deque
from datetime import datetime
from pathlib import Path
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, ConcatDataset

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


## Data Download

In [6]:
!pip3 install kaggle

In [13]:
if Path("cryptorl_data").exists():
    print(f"cryptorl_data already exists, skipping unzip.")
else:
    !unzip /datalab/cryptorl_data.zip -d cryptorl_data
    print("Unzipped.")

Archive:  /datalab/cryptorl_data.zip
   creating: cryptorl_data/archive/
  inflating: cryptorl_data/__MACOSX/._archive  
  inflating: cryptorl_data/archive/XBTUSD_2020-02-07.csv.xz  
  inflating: cryptorl_data/__MACOSX/archive/._XBTUSD_2020-02-07.csv.xz  
  inflating: cryptorl_data/archive/XBTUSD_2020-02-19.csv.xz  
  inflating: cryptorl_data/__MACOSX/archive/._XBTUSD_2020-02-19.csv.xz  
  inflating: cryptorl_data/archive/XBTUSD_2020-01-28.csv.xz  
  inflating: cryptorl_data/__MACOSX/archive/._XBTUSD_2020-01-28.csv.xz  
  inflating: cryptorl_data/archive/XBTUSD_2020-02-15.csv.xz  
  inflating: cryptorl_data/__MACOSX/archive/._XBTUSD_2020-02-15.csv.xz  
  inflating: cryptorl_data/archive/XBTUSD_2020-01-24.csv.xz  
  inflating: cryptorl_data/__MACOSX/archive/._XBTUSD_2020-01-24.csv.xz  
  inflating: cryptorl_data/archive/XBTUSD_2020-02-23.csv.xz  
  inflating: cryptorl_data/__MACOSX/archive/._XBTUSD_2020-02-23.csv.xz  
  inflating: cryptorl_data/archive/XBTUSD_2020-01-12.csv.xz  
  infla

In [15]:
# List files
data_dir = "cryptorl_data"
files = os.listdir(data_dir)
print("Available files:", files)

Available files: ['archive', '__MACOSX']


In [16]:
# List files
data_dir = "cryptorl_data"
files = os.listdir(data_dir)
print("Available files:", files)

Available files: ['archive', '__MACOSX']


## Exploratory Data Analysis

In [17]:
def explore_dataset(data_path):
    """Explore the CryptoRL dataset"""

    # We are interested in the number of files and the content of the dataset
    print("Files in dataset:")
    # archive is the name of the data zip
    archive_path = os.path.join(data_path, 'archive')
    for file in os.listdir(archive_path):
        if file.endswith('.csv.xz'):
            file_path = os.path.join(archive_path, file)
            # Use xz compression for reading
            df = pd.read_csv(file_path, compression='xz')
            print(f"\n {file}:")
            print(f"   Shape: {df.shape}")
            print(f"   Columns: {list(df.columns)}")
            print(f"   Sample data:")
            print(df.head(3))

    # Check for JSON files existent in path
    for file in os.listdir(data_path):
        if file.endswith('.json'):
            with open(os.path.join(data_path, file)) as f:
                data = json.load(f)
                print(f"\n {file} content:")
                print(json.dumps(data, indent=2)[:500] + "...")


# Path to cryptorl data
data_path = "cryptorl_data"

# Find all .csv.xz files in the archive subdirectory
file_pattern = os.path.join(data_path, "archive", "*.csv.xz")
files = glob.glob(file_pattern)
print(f"Found {len(files)} compressed CSV files")
print("Sample files:", files[:5])

# Load a single file to understand structure
first_file = files[0]
df_sample = pd.read_csv(first_file, compression='xz')
print(f"\n Sample from {os.path.basename(first_file)}:")
print(f"Shape: {df_sample.shape}")
print(f"Columns: {df_sample.columns.tolist()}")
print("\nFirst few rows:")
print(df_sample.head())

# Check data types
print("\nData types:")
print(df_sample.dtypes)

explore_dataset("cryptorl_data")

Found 58 compressed CSV files
Sample files: ['cryptorl_data/archive/XBTUSD_2020-01-31.csv.xz', 'cryptorl_data/archive/XBTUSD_2020-02-19.csv.xz', 'cryptorl_data/archive/XBTUSD_2020-01-09.csv.xz', 'cryptorl_data/archive/XBTUSD_2020-02-22.csv.xz', 'cryptorl_data/archive/XBTUSD_2020-02-23.csv.xz']

 Sample from XBTUSD_2020-01-31.csv.xz:
Shape: (86344, 205)
Columns: ['Unnamed: 0', 'midpoint', 'spread', 'buys', 'sells', 'bids_distance_0', 'bids_distance_1', 'bids_distance_2', 'bids_distance_3', 'bids_distance_4', 'bids_distance_5', 'bids_distance_6', 'bids_distance_7', 'bids_distance_8', 'bids_distance_9', 'bids_distance_10', 'bids_distance_11', 'bids_distance_12', 'bids_distance_13', 'bids_distance_14', 'bids_distance_15', 'bids_distance_16', 'bids_distance_17', 'bids_distance_18', 'bids_distance_19', 'bids_notional_0', 'bids_notional_1', 'bids_notional_2', 'bids_notional_3', 'bids_notional_4', 'bids_notional_5', 'bids_notional_6', 'bids_notional_7', 'bids_notional_8', 'bids_notional_9', 'b

The example columns are: midpoint, spread, buys, sells, bids_distance_X, bids_notional_X, bids_cancel_notional_X, bids_limit_notional_X, asks_distance_X.

The few few rows show us the sample data like timestamps and bid/ask order book metrics

In [21]:
def get_dataset_stats(data_path="cryptorl_data"):
    """Get statistics about all files without loading full data"""

    stats = []
    for file in Path(data_path).glob("archive/*.csv.xz"):
        file_size = file.stat().st_size / (1024**2)
        df_sample = pd.read_csv(file, nrows=1)

        stats.append({
            'filename': file.name,
            'size_mb': round(file_size, 2),
            'columns': len(df_sample.columns),
            'approx_rows': 0,
            'column_names': df_sample.columns.tolist()
        })

    # Create summary DataFrame
    stats_df = pd.DataFrame(stats)

    print(" Dataset Summary:")
    print(f"Total files: {len(stats_df)}")
    print(f"Total size: {stats_df['size_mb'].sum():.2f} MB")
    print(f"Total columns: {stats_df['columns'].sum()}")

    if not stats_df.empty:
        print(f"Total size: {stats_df['size_mb'].sum():.2f} MB")
    else:
        print("Total size: 0.00 MB (No files found matching the pattern)")

    print(f"\nSample files:")
    print(stats_df.head())

    return stats_df

# Get stats
stats_df = get_dataset_stats()

 Dataset Summary:
Total files: 58
Total size: 763.49 MB
Total columns: 11890
Total size: 763.49 MB

Sample files:
                   filename  size_mb  columns  approx_rows  \
0  XBTUSD_2020-01-31.csv.xz    12.58      205            0   
1  XBTUSD_2020-02-19.csv.xz    13.23      205            0   
2  XBTUSD_2020-01-09.csv.xz    16.12      205            0   
3  XBTUSD_2020-02-22.csv.xz    11.06      205            0   
4  XBTUSD_2020-02-23.csv.xz    11.20      205            0   

                                        column_names  
0  [Unnamed: 0, midpoint, spread, buys, sells, bi...  
1  [Unnamed: 0, midpoint, spread, buys, sells, bi...  
2  [Unnamed: 0, midpoint, spread, buys, sells, bi...  
3  [Unnamed: 0, midpoint, spread, buys, sells, bi...  
4  [Unnamed: 0, midpoint, spread, buys, sells, bi...  


Time-Series Analysis

In [23]:
df = pd.read_csv("cryptorl_data/archive/XBTUSD_2020-01-01.csv.xz")

print(df.head())
print(df.columns)

candidates = [
    c for c in df.columns
    if any(k in c.lower() for k in [
        "label", "y", "target", "direction", "side", "state", "up", "down"
    ])
]

print("candidates:", candidates)

df[[f"bids_distance_{i}" for i in range(5)]].head()

                        system_time  midpoint  spread    buys     sells  \
0  2019-12-31 20:00:00.279922+00:00   7141.75     0.5     0.0       0.0   
1  2019-12-31 20:00:01.279922+00:00   7141.75     0.5     0.0       0.0   
2  2019-12-31 20:00:02.279922+00:00   7141.75     0.5     0.0       0.0   
3  2019-12-31 20:00:03.279922+00:00   7141.75     0.5  4845.0  118233.0   
4  2019-12-31 20:00:04.279922+00:00   7141.75     0.5     0.0  342744.0   

   bids_distance_0  bids_distance_1  bids_distance_2  bids_distance_3  \
0        -0.000035        -0.000105        -0.000175        -0.000245   
1        -0.000035        -0.000105        -0.000175        -0.000245   
2        -0.000035        -0.000105        -0.000175        -0.000245   
3        -0.000035        -0.000105        -0.000175        -0.000245   
4        -0.000035        -0.000105        -0.000175        -0.000245   

   bids_distance_4  ...  asks_market_notional_10  asks_market_notional_11  \
0        -0.000315  ...          

,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4
0,-0.000035,-0.000105,-0.000175,-0.000245,-0.000315
1,-0.000035,-0.000105,-0.000175,-0.000245,-0.000315
2,-0.000035,-0.000105,-0.000175,-0.000245,-0.000315
3,-0.000035,-0.000105,-0.000175,-0.000245,-0.000315
4,-0.000035,-0.000105,-0.000175,-0.000245,-0.000315


## Data Preprocessing

In [24]:
class LOBWindowMidLabelDataset(Dataset):
    """
    X: (T, 4n)
    y: 0/1/2 (midprice down / stationary / up)
    lazy load window
    """
    def __init__(self, file_path: str,
                 levels: int = 20,
                 window_size: int = 50,
                 horizon: int = 10,
                 alpha: float = 1e-5):
        super().__init__()
        self.file_path = Path(file_path)
        self.levels = levels
        self.window_size = window_size
        self.horizon = horizon
        self.alpha = alpha

        self.col_asks_dist = [f"asks_notional_{i}" for i in range(levels)]
        self.col_asks_notional = [f"asks_limit_notional_{i}" for i in range(levels)]
        self.col_bids_dist = [f"bids_notional_{i}" for i in range(levels)]
        self.col_bids_notional = [f"bids_limit_notional_{i}" for i in range(levels)]
        self.lob_cols = (
            self.col_asks_dist +
            self.col_asks_notional +
            self.col_bids_dist +
            self.col_bids_notional
        )

        df = pd.read_csv(self.file_path)

        missing = [c for c in self.lob_cols + ["midpoint"] if c not in df.columns]
        if missing:
            raise RuntimeError(f"{self.file_path} missing columns: {missing}")


        x = df[self.lob_cols].astype("float32").values   # (N, 4n)
        mid = df["midpoint"].astype("float32").values    # (N,)

        self.x_np = x
        self.mid_np = mid

        N = x.shape[0]
        T = window_size
        h = horizon


        end_indices = []
        labels = []
        for i in range(T - 1, N - h):
            m_now = mid[i]
            m_future = mid[i + h]
            r = (m_future - m_now) / m_now

            if r > alpha:
                label = 2  # up
            elif r < -alpha:
                label = 0  # down
            else:
                label = 1  # stationary

            end_indices.append(i)
            labels.append(label)

        self.end_indices = np.array(end_indices, dtype=np.int64)  # (M,)
        self.y = torch.tensor(labels, dtype=torch.long)           # (M,)

        print(f"[LOBWindowMidLabelDataset] {self.file_path.name}: "
              f"{N} ticks -> {self.end_indices.shape[0]} windows (T={T}), labels in {{0,1,2}}.")

    def __len__(self):
        return self.end_indices.shape[0]

    def __getitem__(self, idx):
        """
          X: (T, 4n)
          y: long
        """
        i = int(self.end_indices[idx])
        T = self.window_size


        window = self.x_np[i - T + 1 : i + 1]   # (T, 4n)
        x_torch = torch.from_numpy(window).float()
        y_torch = self.y[idx]
        return x_torch, y_torch

In [26]:
test = LOBWindowMidLabelDataset(
    'cryptorl_data/archive/XBTUSD_2020-02-29.csv.xz',
    levels=20,
    window_size=80,
    horizon=10,
    alpha=1e-4,
)
x, y = test[0]
print(x.shape, x.dtype, y)

labels = test.y.numpy()
_, counts = np.unique(labels, return_counts=True)
print("Label distribution (0:down,1:flat,2:up):", dict(zip(range(3), counts)))

[LOBWindowMidLabelDataset] XBTUSD_2020-02-29.csv.xz: 86302 ticks -> 86213 windows (T=80), labels in {0,1,2}.
torch.Size([80, 80]) torch.float32 tensor(1)
Label distribution (0:down,1:flat,2:up): {0: np.int64(8977), 1: np.int64(69053), 2: np.int64(8183)}


## Attention Pretrain Model

In [27]:
class AttnLOB(nn.Module):
    def __init__(self, T: int, feature_dim: int,   # feature_dim = 4 * n, 80
                 attn_dim: int = 64, num_heads: int = 10, key_dim: int = 16):

        super().__init__()
        self.T = T
        self.feature_dim = feature_dim

        # first conv
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(1, 2), stride=(1, 2))
        self.conv1a = nn.Conv2d(32, 32, kernel_size=(4, 1), padding="same")
        self.conv1b = nn.Conv2d(32, 32, kernel_size=(4, 1), padding="same")

        # second conv
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(1, 5), stride=(1, 5))
        self.conv2a = nn.Conv2d(32, 32, kernel_size=(4, 1), padding="same")
        self.conv2b = nn.Conv2d(32, 32, kernel_size=(4, 1), padding="same")

        # third conv
        self.conv3 = nn.Conv2d(32, 32, kernel_size=(1, 4))          # channel → 5
        self.conv3a = nn.Conv2d(32, 32, kernel_size=(4, 1), padding="same")
        self.conv3b = nn.Conv2d(32, 32, kernel_size=(4, 1), padding="same")

        # fourth conv
        self.conv4  = nn.Conv2d(32, 32, kernel_size=(1, 5)) # W 5 -> 1

        # Inception module
        # branch 1
        self.inc1_1 = nn.Conv2d(32, 64, kernel_size=(1, 1), padding="same")
        self.inc1_2 = nn.Conv2d(64, 64, kernel_size=(3, 1), padding="same")

        # branch 2
        self.inc2_1 = nn.Conv2d(32, 64, kernel_size=(1, 1), padding="same")
        self.inc2_2 = nn.Conv2d(64, 64, kernel_size=(5, 1), padding="same")

        # branch 3
        self.pool3 = nn.MaxPool2d(kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        self.inc3_1 = nn.Conv2d(32, 64, kernel_size=(1, 1), padding="same")

        # 192 -> 160
        #self.proj = nn.Linear(192, attn_dim)
        self.attn_embed_dim = num_heads * key_dim  # 160
        self.proj_in = nn.Linear(192, self.attn_embed_dim)

        # Multi-head self-attention
        self.attn = nn.MultiheadAttention(
            embed_dim=self.attn_embed_dim,
            num_heads=num_heads,
            batch_first=True,
        )
        # 160 -> 64
        self.proj_out = nn.Linear(self.attn_embed_dim, attn_dim)

        self.leaky = nn.LeakyReLU(0.01)

    def forward(self, x):
        """
        x: (B, T, F)  F = 4 * n
        return:
            emb: (B, 64)
            attn_weights: (B, 1, L)
        """
        B, T, F = x.shape
        assert F == self.feature_dim

        x = x.unsqueeze(1)

        # first conv
        x = self.leaky(self.conv1(x))
        x = self.leaky(self.conv1a(x))
        x = self.leaky(self.conv1b(x))

        # second conv
        x = self.leaky(self.conv2(x))
        x = self.leaky(self.conv2a(x))
        x = self.leaky(self.conv2b(x))

        # third conv
        x = self.leaky(self.conv3(x))
        x = self.leaky(self.conv3a(x))
        x = self.leaky(self.conv3b(x))

        # fourth conv
        x = self.leaky(self.conv4(x))   # W: 5 -> 1

        # Inception
        b1 = self.leaky(self.inc1_1(x))
        b1 = self.leaky(self.inc1_2(b1))

        b2 = self.leaky(self.inc2_1(x))
        b2 = self.leaky(self.inc2_2(b2))

        b3 = self.pool3(x)
        b3 = self.leaky(self.inc3_1(b3))

        x = torch.cat([b1, b2, b3], dim=1)

        # reshape
        x = x.squeeze(-1)           # (B, 192, T')
        x = x.permute(0, 2, 1)      # (B, T', 192)

        x = self.proj_in(x)            # (B, T', 160)

        # attention
        query = x[:, -1:, :]        # (B, 1, 160)
        key = value = x             # (B, T', 160)

        attn_out, attn_weights = self.attn(query, key, value,
                                           need_weights=True)

        # (B, 1, 160) → (B, 160)
        attn_vec = attn_out.squeeze(1)
        # 160 -> 64
        emb = self.proj_out(attn_vec) # (B, 64)

        return emb, attn_weights

In [28]:
"""
Classifier
"""
class AttnLOBPretrain(nn.Module):
    def __init__(self, T: int, feature_dim: int,
                 attn_dim: int = 64, num_heads: int = 10, num_classes: int = 3):
        super().__init__()
        self.encoder = AttnLOB(T, feature_dim, attn_dim, num_heads)
        self.classifier = nn.Linear(attn_dim, num_classes)

    def forward(self, x):
        """
        x: (B, T, F)
        """
        emb, attn_w = self.encoder(x)
        logits = self.classifier(emb)   # (B, 3)
        return logits, emb, attn_w

In [29]:
criterion = nn.CrossEntropyLoss()

def evaluate(model, loader):
    model.eval()
    total = 0
    correct = 0
    loss_sum = 0.0
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)      # (B, T, F)
            yb = yb.to(device)      # (B,)
            logits, _, _ = model(xb)
            loss = criterion(logits, yb)
            loss_sum += loss.item() * yb.size(0)

            preds = logits.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    return loss_sum / total, correct / total

## Pretrain Model Training

In [30]:
T = 50
F_dim = 80   # 4 * n

# full file loader
data_dir = Path("cryptorl_data")
csv_files = sorted(data_dir.glob("XBTUSD_*.csv.xz"))

# testing with smaller set
csv_files = csv_files[:2]
csv_files

[]

In [31]:
model = AttnLOBPretrain(T=T, feature_dim=F_dim,
                        attn_dim=64, num_heads=10,
                        num_classes=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

now = datetime.now()
timestamp = now.strftime("%Y%m%d-%H%M%S")
print(timestamp)
model_path = f"models/attn_lob_pretrain_{timestamp}.pt"
print(model_path)

# run then load
epochs = 1 # ->5
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    total_train = 0

    for p in csv_files:
        print(f"\n=== Epoch {epoch}, training on {p.name} ===")
        dataset_day = LOBWindowMidLabelDataset(
            str(p),
            levels=20,
            window_size=T,
            horizon=10,
            alpha=1e-5,
        )

        num_samples = len(dataset_day)
        split = int(num_samples * 0.8)
        train_set_day, val_set_day = torch.utils.data.random_split(
            dataset_day, [split, num_samples - split]
        )

        train_loader = DataLoader(train_set_day, batch_size=64, shuffle=True)
        val_loader   = DataLoader(val_set_day, batch_size=64, shuffle=False)

        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            optimizer.zero_grad()
            logits, _, _ = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * yb.size(0)
            total_train += yb.size(0)

        val_loss_day, val_acc_day = evaluate(model, val_loader)
        print(f"[{p.name}] val_loss: {val_loss_day:.4f}, val_acc: {val_acc_day:.4f}")

        del dataset_day, train_set_day, val_set_day, train_loader, val_loader
        torch.cuda.empty_cache()

    epoch_train_loss = running_loss / total_train
    print(f"\n=== Epoch {epoch} done, train_loss(avg over days): {epoch_train_loss:.4f} ===")

torch.save(model.state_dict(), model_path)

20251216-022944
models/attn_lob_pretrain_20251216-022944.pt


ZeroDivisionError: float division by zero

## Pretrain Model Evaluation

In [ ]:
# small evaluation out-of-sample
csv_files = sorted(data_dir.glob("XBTUSD_*.csv.xz"))
csv_files = csv_files[-2:] ## TODO - remove
print(csv_files)

In [ ]:
# loading our final pretraining model
model_path = f"models/attn_lob_pretrain.pt"

In [ ]:
model = AttnLOBPretrain(T=T, feature_dim=F_dim,
                        attn_dim=64, num_heads=10,
                        num_classes=3).to(device)
state = torch.load(model_path, map_location=device)

model.load_state_dict(state)
model.eval()

global_loss_sum = 0.0
global_correct = 0
global_total = 0

for p in csv_files:
    print(f"\n=== Evaluating on {p.name} ===")
    dataset = LOBWindowMidLabelDataset(
        str(p),
        levels=20,
        window_size=T,
        horizon=10,
        alpha=1e-4,
    )
    loader = DataLoader(dataset, batch_size=256, shuffle=False)

    val_loss, val_acc = evaluate(model, loader)
    print(f"[{p.name}] loss = {val_loss:.4f}, acc = {val_acc:.4f}")


    n = len(dataset)

    global_loss_sum += val_loss * n
    global_correct += val_acc * n
    global_total += n

overall_loss = global_loss_sum / global_total
overall_acc  = global_correct / global_total

print("\n=== Overall evaluation across all days ===")
print(f"overall loss = {overall_loss:.4f}, overall acc = {overall_acc:.4f}")

## Encoder

In [ ]:
class CFG:
    file_path: str = "cryptorl_data/XBTUSD_2020-02-29.csv.xz"
    levels: int = 20
    window_size: int = 50
    horizon: int = 10
    alpha: float = 1e-4

    # RL
    episode_len: int = 2000        # steps per episode
    gamma: float = 0.99
    lr: float = 1e-3
    batch_size: int = 128
    replay_size: int = 50_000
    warmup_steps: int = 2_000      # collect random experience first
    total_steps: int = 30_000      # increase later
    target_update_every: int = 500 # steps

    # Exploration
    eps_start: float = 1.0
    eps_end: float = 0.05
    eps_decay_steps: int = 20_000

    # Reward shaping
    inv_penalty: float = 1e-6      # λ * inv^2
    turn_cost: float = 5e-6        # c * |Δinv|
    inv_max: int = 1               # increase later

cfg = CFG()

In [ ]:
encoder = model.encoder.to(device).eval()

use_raw_features = False
for p in encoder.parameters():
    p.requires_grad = False

In [ ]:
ds = LOBWindowMidLabelDataset(
    cfg.file_path,
    levels=cfg.levels,
    window_size=cfg.window_size,
    horizon=cfg.horizon,
    alpha=cfg.alpha,
)
print(ds)

# Sanity checks
x0, y0 = ds[0]
print("sample x:", x0.shape, x0.dtype, "y:", y0.item())
print("mid array exists:", hasattr(ds, "mid_np"), "end_indices exists:", hasattr(ds, "end_indices"))

In [ ]:
def encode_window(window_txf: torch.Tensor) -> torch.Tensor:
    """
    window_txf: (T, F) torch.float32 on CPU
    returns embedding: (E,) on CPU
    """
    if use_raw_features:
        # simple fallback embedding: mean over time => (F,)
        return window_txf.mean(dim=0)

    # encoder expects (B, T, F)
    x = window_txf.unsqueeze(0).to(device)  # (1,T,F)
    out = encoder(x)

    if isinstance(out, tuple):
        emb = out[0]  # (1,E)
    else:
        emb = out
    return emb.squeeze(0).detach().cpu()

# Determine embedding dim E
tmp_emb = encode_window(ds[0][0])
emb_dim = int(tmp_emb.shape[0])
print("Embedding dim:", emb_dim)

## RL Environment

In [ ]:
class LOBEnv:
    """
    Discrete actions: 0=SHORT, 1=FLAT, 2=LONG
    Inventory is forced to {-inv_max,...,+inv_max} (here inv_max=1).

    State = [embedding, inventory, time_frac]
    Reward = inv_new * (mid_{t+1} - mid_t) - inv_penalty * inv_new^2 - turn_cost * |inv_new - inv_old|
    """
    ACTION_TO_INV = {0: -1, 1: 0, 2: +1}

    def __init__(self, dataset, episode_len=2000, inv_penalty=1e-6, turn_cost=5e-6, inv_max=1):
        self.ds = dataset
        self.episode_len = episode_len
        self.inv_penalty = inv_penalty
        self.turn_cost = turn_cost
        self.inv_max = inv_max

        # We'll step through ds.end_indices; each corresponds to window ending at i
        self.M = len(dataset)
        self.reset()

    def reset(self):
        # start somewhere that allows episode_len steps
        max_start = self.M - (self.episode_len + 2)
        self.pos = random.randint(0, max(0, max_start))
        self.t = 0
        self.inv = 0
        self.prev_inv = 0
        return self._get_obs()

    def _get_mid(self, idx_in_M: int) -> float:
        # idx_in_M maps to end_index i in original ticks
        i = int(self.ds.end_indices[idx_in_M])
        return float(self.ds.mid_np[i])

    def _get_obs(self):
        window, _ = self.ds[self.pos]  # window: (T,F)
        emb = encode_window(window)    # (E,)
        time_frac = torch.tensor([self.t / max(1, self.episode_len)], dtype=torch.float32)
        inv_feat = torch.tensor([float(self.inv)], dtype=torch.float32)
        obs = torch.cat([emb.float(), inv_feat, time_frac], dim=0)  # (E+2,)
        return obs

    def step(self, action: int):
        action = int(action)
        target_inv = self.ACTION_TO_INV[action] * self.inv_max

        self.prev_inv = self.inv
        self.inv = int(np.clip(target_inv, -self.inv_max, self.inv_max))

        mid_t = self._get_mid(self.pos)
        mid_tp1 = self._get_mid(self.pos + 1)

        dmid = mid_tp1 - mid_t
        reward = (self.inv * dmid) - (self.inv_penalty * (self.inv ** 2)) - (self.turn_cost * abs(self.inv - self.prev_inv))

        self.pos += 1
        self.t += 1

        done = (self.t >= self.episode_len) or (self.pos >= self.M - 2)
        obs = self._get_obs() if not done else None

        info = {"mid_t": mid_t, "mid_tp1": mid_tp1, "dmid": dmid, "inv": self.inv}
        return obs, float(reward), done, info

In [ ]:
class ReplayBuffer:
    def __init__(self, capacity: int):
        self.buf = deque(maxlen=capacity)

    def push(self, s, a, r, sp, done):
        self.buf.append((s, a, r, sp, done))

    def sample(self, batch_size: int):
        batch = random.sample(self.buf, batch_size)
        s, a, r, sp, done = zip(*batch)
        return (
            torch.stack(s),
            torch.tensor(a, dtype=torch.long),
            torch.tensor(r, dtype=torch.float32),
            torch.stack([x if x is not None else torch.zeros_like(s[0]) for x in sp]),
            torch.tensor(done, dtype=torch.float32),
        )

    def __len__(self):
        return len(self.buf)

## Q Network

In [ ]:
class QNet(nn.Module):
    def __init__(self, in_dim: int, n_actions: int = 3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, n_actions),
        )

    def forward(self, x):
        return self.net(x)

## RL Train

In [ ]:
env = LOBEnv(
    ds,
    episode_len=cfg.episode_len,
    inv_penalty=cfg.inv_penalty,
    turn_cost=cfg.turn_cost,
    inv_max=cfg.inv_max
)

obs0 = env.reset()
state_dim = obs0.shape[0]
print("State dim (E+2):", state_dim)

In [ ]:
def epsilon_by_step(step: int, cfg: CFG) -> float:
    if step >= cfg.eps_decay_steps:
        return cfg.eps_end
    frac = step / cfg.eps_decay_steps
    return cfg.eps_start + frac * (cfg.eps_end - cfg.eps_start)

@torch.no_grad()
def select_action(qnet: nn.Module, obs: torch.Tensor, eps: float) -> int:
    if random.random() < eps:
        return random.randint(0, 2)
    q = qnet(obs.unsqueeze(0).to(device))  # (1,3)
    return int(torch.argmax(q, dim=1).item())

@torch.no_grad()
def evaluate_policy(env: LOBEnv, qnet: nn.Module, episodes: int = 3):
    qnet.eval()
    rets = []
    for _ in range(episodes):
        obs = env.reset()
        done = False
        total = 0.0
        while not done:
            a = select_action(qnet, obs, eps=0.0)
            obs, r, done, info = env.step(a)
            total += r
        rets.append(total)
    qnet.train()
    return float(np.mean(rets)), float(np.std(rets))

In [ ]:
qnet = QNet(state_dim, n_actions=3).to(device)
tgt = QNet(state_dim, n_actions=3).to(device)
tgt.load_state_dict(qnet.state_dict())
tgt.eval()

opt = torch.optim.Adam(qnet.parameters(), lr=cfg.lr)
rb = ReplayBuffer(cfg.replay_size)

obs = env.reset()
episode_return = 0.0
episode_returns = []
eps_log = []
train_step_log = []
loss_log = []
eval_step_log = []
eval_mean_log = []
eval_std_log = []
for step in range(1, cfg.total_steps + 1):
    eps = epsilon_by_step(step, cfg)
    eps_log.append(eps)

    a = select_action(qnet, obs, eps)
    obs2, r, done, info = env.step(a)

    rb.push(obs, a, r, (obs2 if obs2 is not None else torch.zeros_like(obs)), done)
    obs = obs2 if obs2 is not None else env.reset()

    episode_return += r
    if done:
        episode_returns.append(episode_return)
        episode_return = 0.0

    # learn
    if len(rb) >= cfg.warmup_steps and len(rb) >= cfg.batch_size:
        s, a_b, r_b, sp, done_b = rb.sample(cfg.batch_size)
        s = s.to(device)
        sp = sp.to(device)
        a_b = a_b.to(device)
        r_b = r_b.to(device)
        done_b = done_b.to(device)

        q = qnet(s).gather(1, a_b.view(-1,1)).squeeze(1)

        with torch.no_grad():
            # standard DQN target (you can switch to Double DQN later)
            q_next = tgt(sp).max(dim=1).values
            target = r_b + cfg.gamma * (1.0 - done_b) * q_next

        loss = F.smooth_l1_loss(q, target)
        loss_log.append(loss.item())
        train_step_log.append(step)

        opt.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(qnet.parameters(), 1.0)
        opt.step()

    # target update
    if step % cfg.target_update_every == 0:
        tgt.load_state_dict(qnet.state_dict())

    # logging
    if step % 1000 == 0:
        avg_ep = np.mean(episode_returns[-10:]) if len(episode_returns) else float("nan")
        eval_mean, eval_std = evaluate_policy(env, qnet, episodes=3)
        eval_step_log.append(step)
        eval_mean_log.append(eval_mean)
        eval_std_log.append(eval_std)
        print(f"step={step:6d} eps={eps:.3f} buffer={len(rb):5d} "
              f"train_ep_avg(last10)={avg_ep:.6f} eval_return={eval_mean:.6f}±{eval_std:.6f}")

print("Done.")

In [ ]:
# Episode returns (training)
plt.figure()
plt.plot(episode_returns)
plt.xlabel("Episode")
plt.ylabel("Return")
plt.title("Training episode returns")
plt.show()

# Eval return vs steps
plt.figure()
plt.plot(eval_step_log, eval_mean_log)
plt.xlabel("Env step")
plt.ylabel("Eval return (mean)")
plt.title("Evaluation return vs training steps")
plt.show()

# Loss curve (training updates)
plt.figure()
plt.plot(train_step_log, loss_log)
plt.xlabel("Env step")
plt.ylabel("SmoothL1 loss")
plt.title("DQN loss over time")
plt.show()

# Epsilon schedule
plt.figure()
plt.plot(eps_log)
plt.xlabel("Env step")
plt.ylabel("Epsilon")
plt.title("Exploration schedule")
plt.show()